## OpenCV

In [2]:
import time

import cv2
import numpy as np

import torch

import mmcv
from mmcv import imread
import mmengine
from mmengine.registry import init_default_scope

from mmpose.apis import inference_topdown
from mmpose.apis import init_model as init_pose_estimator
from mmpose.evaluation.functional import nms

from mmdet.apis import inference_detector, init_detector

In [3]:
# 有 GPU 就用 GPU，没有就用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


## 准备好模型文件

In [4]:
## 目标检测模型

# data/rtmdet_tiny_triangle.py
# checkpoint/rtmdet_tiny_triangle_epoch_200_202305120847-3cd02a8f.pth

## 关键点检测模型

# data/rtmpose-s-triangle.py
# checkpoint/rtmpose-s-triangle-300-32642023_20230524.pth

## 构建目标检测模型

In [5]:
# RTMDet 三角板检测
detector = init_detector(
    'data/rtmdet_tiny_triangle.py',
    'checkpoint/rtmdet_tiny_triangle_epoch_200_202305120847-3cd02a8f.pth',
    device=device
)

Loads checkpoint by local backend from path: checkpoint/rtmdet_tiny_triangle_epoch_200_202305120847-3cd02a8f.pth


## 构建关键点检测模型

In [6]:
pose_estimator = init_pose_estimator(
    'data/rtmpose-s-triangle.py',
    'checkpoint/rtmpose_s_triangle_300-34bfaeb2_20230512.pth',
    device=device,
    cfg_options={'model': {'test_cfg': {'output_heatmaps': True}}}
)

Loads checkpoint by local backend from path: checkpoint/rtmpose_s_triangle_300-34bfaeb2_20230512.pth
The model and loaded state dict do not match exactly

size mismatch for backbone.stem.0.conv.weight: copying a param with shape torch.Size([24, 3, 3, 3]) from checkpoint, the shape in current model is torch.Size([16, 3, 3, 3]).
size mismatch for backbone.stem.0.bn.weight: copying a param with shape torch.Size([24]) from checkpoint, the shape in current model is torch.Size([16]).
size mismatch for backbone.stem.0.bn.bias: copying a param with shape torch.Size([24]) from checkpoint, the shape in current model is torch.Size([16]).
size mismatch for backbone.stem.0.bn.running_mean: copying a param with shape torch.Size([24]) from checkpoint, the shape in current model is torch.Size([16]).
size mismatch for backbone.stem.0.bn.running_var: copying a param with shape torch.Size([24]) from checkpoint, the shape in current model is torch.Size([16]).
size mismatch for backbone.stem.1.conv.weight:

In [7]:
# 这段代码的目的是确保在没有 GPU 支持的情况下，pose_estimator 模型能够正确运行
from mmengine.model.utils import revert_sync_batchnorm
if not torch.cuda.is_available():
    pose_estimator = revert_sync_batchnorm(pose_estimator)

In [8]:
# 该函数来源于mmengine，这些参数都是被人为定义好的
# 这里的意思是目标检测框基本的anchor box大小源自mmdet中的default_scope配置文件
init_default_scope(detector.cfg.get('default_scope', 'mmdet'))

10/08 22:41:49 - mmengine - WARNING - The current default scope "mmpose" is not "mmdet", `init_default_scope` will force set the currentdefault scope to "mmdet".


## 数据集元数据

In [9]:
# 三角板关键点检测数据集-元数据（直接从config配置文件中粘贴）
# 可以理解为pose_estimator推理出的信息做对应可视化的属性，该对象定义了不同关键点的颜色，不同关键点之间相连的线颜色
dataset_info = {
    'keypoint_info':{
        0:{'name':'angle_30','id':0,'color':[255,0,0],'type': '','swap': ''},
        1:{'name':'angle_60','id':1,'color':[0,255,0],'type': '','swap': ''},
        2:{'name':'angle_90','id':2,'color':[0,0,255],'type': '','swap': ''}
    },
    'skeleton_info': {
        0: {'link':('angle_30','angle_60'),'id': 0,'color': [100,150,200]},
        1: {'link':('angle_60','angle_90'),'id': 1,'color': [200,100,150]},
        2: {'link':('angle_90','angle_30'),'id': 2,'color': [150,120,100]}
    }
}

In [10]:
# 关键点类别和关键点ID的映射字典
label2id = {}
for each in dataset_info['keypoint_info'].items():
    label2id[each[1]['name']] = each[0]

## 可视化配置

In [17]:
# 检测框的颜色
bbox_color = (150,0,0)
# 检测框的线宽
bbox_thickness = 3
# 关键点半径
kpt_radius = 10
# 连接线宽
skeleton_thickness = 6

# 目标检测置信度阈值
CONF_THRES = 0.55

## 逐帧处理函数

In [20]:
# 处理帧函数
def process_frame(img):
    '''输入BGR格式的 numpy array，输出BGR格式的 numpy array'''
    
    # 记录该帧开始处理的时间
    start_time = time.time()
    
    # 逐帧处理操作

    # 获取目标检测预测结果
    # init_default_scope(detector.cfg.get('default_scope', 'mmdet')) # 我觉得之前有了加载配置，这里多余了
    detect_result = inference_detector(detector, img) # 进行推理
    pred_instance = detect_result.pred_instances.cpu().numpy() # 转换一下推理结果类型

    # 对推理结果做一个筛选，选出置信度较高的一些框
    bboxes = np.concatenate((pred_instance.bboxes, pred_instance.scores[:, None]), axis=1)
    bboxes = bboxes[np.logical_and(pred_instance.labels == 0, pred_instance.scores > CONF_THRES)]
    bboxes = bboxes[nms(bboxes, 0.3)][:, :4].astype('int')
    
    # 获取每个 bbox 的关键点预测结果
    pose_results = inference_topdown(pose_estimator, img, bboxes)
    keypoints_all = []
    # 将每个框的所有关键点为一组依次存入keypoints_all
    for each in pose_results:
        keypoints_all.append(each.pred_instances.keypoints[0].astype('int').tolist())

    # 在每张图片上的目标检测框中画出各关键点
    for bbox_idx, bbox in enumerate(bboxes): # 遍历每个检测框

        # 画框
        img = cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), bbox_color, bbox_thickness)

        # 索引为 bbox_idx 的框，每个关键点的坐标
        keypoints = keypoints_all[bbox_idx]

        # 画连线
        for skeleton_id, skeleton in dataset_info['skeleton_info'].items(): # 遍历每一种连接
            skeleton_color = skeleton['color']
            srt_kpt_id = label2id[skeleton['link'][0]] # 起始点的类别 ID
            srt_kpt_xy = keypoints[srt_kpt_id]         # 起始点的 XY 坐标
            dst_kpt_id = label2id[skeleton['link'][1]] # 终止点的类别 ID
            dst_kpt_xy = keypoints[dst_kpt_id]         # 终止点的 XY 坐标
            img = cv2.line(img, (srt_kpt_xy[0],srt_kpt_xy[1]),(dst_kpt_xy[0],dst_kpt_xy[1]),color=skeleton_color,thickness=skeleton_thickness)

        # 画关键点
        for kpt_idx, kpt_xy in enumerate(keypoints): # 遍历该检测框中的每一个关键点
            kpt_color = dataset_info['keypoint_info'][kpt_idx]['color']
            img = cv2.circle(img, (kpt_xy[0], kpt_xy[1]), kpt_radius, kpt_color, -1)
    
    # 记录该帧处理完毕的时间
    end_time = time.time()
    # 计算每秒处理图像帧数FPS
    FPS = 1/(end_time - start_time)

    # 在画面上写字：图片，字符串，左上角坐标，字体，字体大小，颜色，字体粗细
    scaler = 1 # 文字大小
    FPS_string = 'FPS  '+str(int(FPS)) # 写在画面上的字符串
    img = cv2.putText(img, FPS_string, (25 * scaler, 100 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)

    return img

## 获取摄像头

In [22]:
# ubuntu应该可以，windows因安装环境有各种问题
# 调用摄像头逐帧实时处理模板
# 不需修改任何代码，只需修改process_frame函数即可
# 按q键或Esc键关闭画面

# 导入opencv-python
import cv2
import time

# 获取摄像头，传入0表示获取系统默认摄像头
cap = cv2.VideoCapture(1)

# 打开cap
cap.open(0)

# 无限循环，直到break被触发
while cap.isOpened():
    
    # 获取画面
    success, frame = cap.read()
    
    if not success: # 如果获取画面不成功，则退出
        print('获取画面不成功，退出')
        break
    
    ## 逐帧处理
    frame = process_frame(frame)
    
    # 展示处理后的三通道图像
    cv2.imshow('my_window',frame)
    
    key_pressed = cv2.waitKey(60) # 每隔多少毫秒毫秒，获取键盘哪个键被按下
    # print('键盘上被按下的键：', key_pressed)

    if key_pressed in [ord('q'),27]: # 按键盘上的q或esc退出（在英文输入法下）
        break
    
# 关闭摄像头
cap.release()

# 关闭图像窗口
cv2.destroyAllWindows()

KeyError: 'PackDetInputs is not in the transform registry. Please check whether the value of `PackDetInputs` is correct or it was registered as expected. More details can be found at https://mmengine.readthedocs.io/en/latest/advanced_tutorials/config.html#import-the-custom-module'